In [75]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

DEBUG = True

In [87]:
if (DEBUG):
    train = pd.read_json('./train.json')
    test = pd.read_json('./test.json')
else:
    train = pd.read_json('../input/train.json')
    test = pd.read_json('../input/test.json')
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [89]:
class FuncCreator:
    def __init__(self):
        self.dict_cuisine = dict()
        self.ind = 0
    def __call__(self, it):
        x = it.lower()
        if not (x in self.dict_cuisine.keys()):
            self.dict_cuisine[x] = self.ind
            self.ind = self.ind + 1
        return self.dict_cuisine[x]
    
    def getNameFromId(self, id):
        for key, val in self.dict_cuisine.items():
            if (val == id):
                return key
        return "UNKNOW"
    

        

rep = FuncCreator()

train.loc[:,'cuisine'] = train.loc[:,'cuisine'].apply(rep)

#test.loc[:'cuisine'] = test.loc[:, 'cuisine'].apply(rep)



In [90]:
class FindALL:
    def __init__(self):
        self.ingr = dict()
        self.ind = 0
        self.lock=False
    def __call__(self, x):
        res=''
        for item0 in x:
            item = item0.lower()
            if not item in self.ingr.keys():
                if self.lock:
                    self.ingr[item] = self.ind-100
                else:
                    self.ingr[item] = self.ind
                    self.ind = self.ind + 1
            res = res + str(self.ingr[item]) + " "
        return res
    
    def setlock(self):
        self.lock = True

finder = FindALL()

train.loc[:,'ingredients'] = train.loc[:,'ingredients'].apply(finder)
#print(train)

finder.setlock()

test.loc[:,'ingredients'] = test.loc[:,'ingredients'].apply(finder)

In [79]:
#print(finder.ingr)
#print(train)

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None,\
                             max_features = 50000, binary = False) 
vectorizer.fit(train['ingredients'])
bagOfWords = vectorizer.transform(train['ingredients'])
vocabulary = vectorizer.get_feature_names()


#vectorizer2 = CountVectorizer()
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None,\
#                             max_features = 50000, binary = True) 
bagOfWords2 = vectorizer.transform(test['ingredients'])
#vocabulary2 = vectorizer2.get_feature_names()

In [81]:
#print(bagOfWords)

In [114]:
from sklearn import *
if (DEBUG):
    end = 36000
    size = bagOfWords.getnnz()
    bagOfWordsTrain = bagOfWords[0:end,:]
    bagOfWordsTest = bagOfWords[end:size,:]

    i = 0.01
    step = 0.02
    maxval = 2.0
    
    imax=0
    mval = 0
    
    #SVM = svm.LinearSVR(C=0.101, fit_intercept=True)
    #SVM.fit(bagOfWordsTrain, train.iloc[0:end]['cuisine'])
    #jj = SVM.score(bagOfWordsTest, train.iloc[end:size]['cuisine'])
    #print(jj)
    
    #"""
    while (i < maxval):
        #SVM = svm.LinearSVC(C=i, fit_intercept=True)
        SVM = svm.LinearSVC(C=i, fit_intercept=True)
        SVM.fit(bagOfWordsTrain, train.iloc[0:end]['cuisine'])
        jj = SVM.score(bagOfWordsTest, train.iloc[end:size]['cuisine'])
        
        if (mval < jj):
            mval = jj
            imax = i
            print(i, jj)
            print("\n")
            
        SVM = svm.LinearSVC(C=i,fit_intercept=False)
        SVM.fit(bagOfWordsTrain, train.iloc[0:end]['cuisine'])
        jj = SVM.score(bagOfWordsTest, train.iloc[end:size]['cuisine'])
        
        if (mval < jj):
            mval = jj
            imax = i
            print(i, jj, "False")
            print("\n")
            
        i=i+step

        #print(jj)"""
    
else:
    size = bagOfWords.getnnz()

    SVM = svm.LinearSVC(C=0.101, fit_intercept=True)
    SVM.fit(bagOfWords, train.iloc[0:size]['cuisine'])


0.01 0.762586115527292


0.03 0.78404875463699


0.05 0.7898781134075251


0.09000000000000001 0.7901430842607313




KeyboardInterrupt: 

In [88]:
#end = 36
#0.7708002119766826
#0.7710651828298887
#0.7723900370959195
#0.7731849496555379
#0.09000000000000001 0.7901430842607313
#0.07300000000000005 0.7904080551139374
#0.10100000000000008 0.7912029676735559

print('wwwAAA'.lower())

wwwaaa


In [41]:
#print(test)
res = SVM.predict(bagOfWords2)
print(res)
print(res[1])

[9 1 6 ... 6 1 7]
1


In [33]:
print(rep.getNameFromId(9))
#print(test)

british


In [46]:
f =open("svm_output.csv", "w")
f.write("id,cuisine\n")
i = 0
for item in test["id"]:
    f.write(str(item))
    f.write(",")
    f.write(rep.getNameFromId(res[i]))
    f.write("\n")
    i= i +1
    
    
f.close()